In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

In [17]:
counties = gpd.read_file("tl_2024_us_county.zip")
nc_counties = counties[counties['STATEFP'] == '37']

In [18]:
nc_counties

,STATEFP,COUNTYFP,COUNTYNS,GEOID,GEOIDFQ,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
23,37,037,01008544,37037,0500000US37037,Chatham,Chatham County,06,H1,G4020,450,20500,None,A,1765030860,70580568,+35.7049939,-079.2514542,"POLYGON ((-79.241 35.571, -79.241 35.571, -79...."
59,37,001,01008531,37001,0500000US37001,Alamance,Alamance County,06,H1,G4020,268,15500,None,A,1096763237,27913357,+36.0439535,-079.4005733,"POLYGON ((-79.268 35.909, -79.268 35.909, -79...."
90,37,057,01008548,37057,0500000US37057,Davidson,Davidson County,06,H1,G4020,268,49180,None,A,1432251776,37596316,+35.7951312,-080.2071070,"POLYGON ((-80.394 35.973, -80.394 35.973, -80...."
181,37,069,01008553,37069,0500000US37069,Franklin,Franklin County,06,H1,G4020,450,39580,None,A,1273761678,7173998,+36.0882466,-078.2830821,"POLYGON ((-78.13 36.24, -78.13 36.239, -78.13 ..."
218,37,155,01026130,37155,0500000US37155,Robeson,Robeson County,06,H1,G4020,246,31300,None,A,2453480732,5076395,+34.6392096,-079.1008811,"POLYGON ((-79.128 34.883, -79.128 34.883, -79...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3113,37,175,01008589,37175,0500000US37175,Transylvania,Transylvania County,06,H1,G4020,120,14820,None,A,979943598,5091023,+35.2101025,-082.8166660,"POLYGON ((-82.798 35.361, -82.798 35.362, -82...."
3114,37,171,01008587,37171,0500000US37171,Surry,Surry County,06,H1,G4020,268,34340,None,A,1379541809,10392704,+36.4154160,-080.6864631,"POLYGON ((-80.446 36.455, -80.446 36.448, -80...."
3156,37,043,01008546,37043,0500000US37043,Clay,Clay County,06,H1,G4020,None,None,None,A,556849483,15046554,+35.0529971,-083.7522638,"POLYGON ((-83.866 35.127, -83.865 35.128, -83...."
3165,37,055,01026133,37055,0500000US37055,Dare,Dare County,06,H1,G4020,545,28620,None,A,992558552,3000513281,+35.6062688,-075.7675364,"POLYGON ((-75.813 36.224, -75.793 36.228, -75...."
